In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# import all the packages
from bs4 import BeautifulSoup
import requests

In [3]:
## Create a list of years which you have data for
years = [str(x) for x in range(1965,2016)]

In [4]:
# The url you need to append the data to.
wiki_url = 'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_'

In [5]:
# Hit all the year links and get the html content and put em in a dictionary where years are your keys
def hit_em_all():
    year_wise_html = {}
    for year in years:
        url = wiki_url+year
        result = requests.get(url)
        content = result.content
        year_wise_html[year] = BeautifulSoup(content, 'html.parser')
    return(year_wise_html)

In [6]:
# Call em all
hot_100s = hit_em_all()

In [7]:
# Identify the table containing the hot 100 from each html (of the respective year)
def get_song_list_table(table):
    table_class = table.attrs['class']
    table_class_list = []
    for class_name in table_class:
        #print(class_name)
        # this extracts the class name of the table. It is of type unicode.
        # Hence, it has to be encoded using ascii to convert to a String
        table_class_list.append(class_name.encode('ascii','ignore'))
    class_name = " ".join(table_class_list)
    return(class_name)

In [8]:
# from all the tables in the page, find out the one with the class name 'wikitable sortable'
def get_the_table(page):
    tables = page.findChildren('table')
    for table in tables:
        if get_song_list_table(table) == 'wikitable sortable':
            return(table)

In [9]:
# Create a dictionary for each year where year is the key and the table becomes the value
def get_all_year_song_tables():
    year_wise_table = {}
    for year, page in hot_100s.items():
        year_wise_table[year] = get_the_table(page)
    return(year_wise_table)

In [10]:
year_wise_table = get_all_year_song_tables()

In [11]:
len(year_wise_table)

51

In [12]:
# New Datapipeline
# Once you have the table, you need to push each of them into our data pipeline and extract the 
# song and artist links.. and then hit the song links and extract the table containing the genre info/
# and finally extract the name  of the genres.
def data_pipeline(year):
    # get year as an input and convert that to string
    year = ''+str(year)
    # index your dictionary with the input year and extract all td elements from the table
    all_tds = year_wise_table[year].find_all('td')

    my_tds = []
    for td in all_tds:
        # if the text of the td element is a number, ignore it
        if not td.text.isdigit():
            # the rest you append to our list
            my_tds.append(td)

    # artist links and song links are alternatively placed as td elements. So take them apart!
    artist_links = my_tds[1::2]
    song_links = my_tds[::2]

    # Get links to the  artists
    def get_full_artist_links():
        all_artists = []
        for artist in artist_links:
            if not artist.text.isdigit():
                all_artists.append(artist.text.encode('ascii','ignore'))
        return(all_artists)

    artists = get_full_artist_links()

    # Get song links along with song names
    def get_full_song_links():
        full_song_links = []
        song_name_list = []
        wikipedia = 'https://en.wikipedia.org'
        for song in song_links:
            #print(song)
            # song info is embedded inside anchor tags in the td elements
            a = song.find('a')
            #print(a)
            if a:
                # get the href element from your anchor tags
                link = a.attrs['href']
                # get the text from the anchor tag.. which is your song's name
                song_name_list.append(a.text.encode('ascii','ignore'))
                # extract the link from the href tag and append it to the wiki url
                full_song_links.append(wikipedia+link.encode('ascii','ignore'))
        return(full_song_links,song_name_list)

    links, songs = get_full_song_links()

    # Now, we have three lists -> songs (name of songs), artists(name of artists), links(links to the songs)
    # We are gonna create a df out of them with year as the first column.
    # We are gonna create a year list with the length of the smallest list out of artists, songs and links
    table = pd.DataFrame(list(zip([1990]*min(len(artists),len(songs),len(links)),songs, artists, links)), columns=['Year','Song','Artist','Song Link'])

    table.head(5)
    
    # Get the table containing the genre info
    def get_the_right_table(table):
        attributes = table.attrs
        table_class_list = []
        class_name=''
        if 'class' in attributes:
            table_classes = table.attrs['class']
            for class_name in table_classes:
                table_class_list.append(class_name.encode('ascii','ignore'))
            class_name = " ".join(table_class_list)
            return(class_name)

    
    def get_genres(table):
        genre_dict = dict()
        songs = []
        for index, row in table.iterrows():
            #print('Song',row['Song'])
            #print('Artist',row['Artist'])
            try:
                # hit the song links and get the html content
                result = requests.get(row['Song Link'])
                content = result.content
                html = BeautifulSoup(content, 'html.parser')
                # get all tables from the  html
                tables = html.findChildren('table')
                my_table = ''
                for table in tables:
                    # if the table name is 'infobox vevent', then thats the one that contains the genre info
                    if get_the_right_table(table) == "infobox vevent":
                        my_table = table
                        # one page could have multiple tables with the same class name.
                        # cos many artists have covered the same song in different styles.
                        # Hence, we need to get rid of those genres and stick with the original genre info
                        # So we are breaking the loop here cos we are not gonna look
                        # at any more tables (if any), to get more genre info
                        break
                # from that table, get all anchor tags
                # from that table, get all anchor tags
                all_as = my_table.find('td',{'class':'category hlist'})
                a = all_as.text.encode('ascii','ignore')
                # remove square brackets and digits if any
                pattern = r'\[.*?\]'
                s = re.sub(pattern, '', a)
                genres = re.sub(r'\d+', '', s)
                #print(genres)
                key = row['Song']
                genre_dict[key] = genres

            except Exception:
                pass
        return(genre_dict)

    #if hasNumbers(a.text.encode('ascii','ignore'))
    genre_dict = get_genres(table)

    #print(genre_dict)

    # Now, we have the genre dictionary, extract them all and split list of lists into seperate items.

    # Create a df with songs and genres lists
    df_genres = pd.DataFrame(list(genre_dict.iteritems()),
                      columns=['Song','Genres'])

    #print(df_genres.head(5))

    return(df_genres)

In [13]:
year = range(1965,2016)

In [ ]:
# Automate the entire thing.
for y in year:
    df = data_pipeline(y)
    # Write the output to csv file for each year.
    a = str(y)+'.csv'
    df.to_csv(a)